In [1]:
!pip install yfinance numpy pandas seabron scikit-learn

ERROR: Could not find a version that satisfies the requirement seabron (from versions: none)
ERROR: No matching distribution found for seabron


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 10px;
">

<b>Ativo de Teste — Itaú (Mapeamento Inicial de Features)</b><br>
Selecionado como ativo de referência para validação e mapeamento completo das features iniciais do modelo.  
Todos os testes a seguir servirão de base para o **Modelo A**, no qual essas etapas serão automatizadas, incluindo **engenharia de dados** e **tratamento de features**.

<br><br>
<b>I) Dados de Preço e Volume</b>
<ul>
  <li>Preço de Abertura (<i>Open</i>)</li>
  <li>Preço de Fechamento (<i>Close</i>)</li>
  <li>Máxima (<i>High</i>)</li>
  <li>Mínima (<i>Low</i>)</li>
  <li>Volume</li>
  <li>Amplitude Diária (<i>Daily Range</i>)</li>
  <li>Corpo Real (<i>Real Body</i>)</li>
  <li>Sombras Superior e Inferior (<i>Upper / Lower Shadow</i>)</li>
  <li>Log-Retorno (<i>t</i>, <i>t−1</i>)</li>
  <li>Janelas Deslizantes — 14, 16, 30 e 60 períodos</li>
</ul>

<b>II) Indicadores Técnicos</b>
<ul>
  <li>Média Móvel Simples (SMA — 7 e 21)</li>
  <li>Média Móvel Exponencial (EMA — 12 e 26)</li>
  <li>RSI (7 e 14)</li>
  <li>MACD (12, 26 e Sinal)</li>
  <li>Bandas de Bollinger (21 períodos)</li>
  <li>ATR (7 e 14)</li>
  <li>High–Low Spread</li>
  <li>Squeeze Momentum (<i>SQZ</i>)</li>
</ul>

</div>


In [3]:
DATA_INICIAL = '2019-04-09'
DATA_FIM = '2025-10-01'

In [4]:
ticker = "ITUB4.SA"
itau = yf.download(ticker, start=DATA_INICIAL, end=DATA_FIM)
print(itau.head())
if isinstance(itau.columns, pd.MultiIndex):
    itau.columns = itau.columns.get_level_values(0)
itau.columns

C:\Users\alexi\AppData\Local\Temp\ipykernel_5080\2117986326.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  itau = yf.download(ticker, start=DATA_INICIAL, end=DATA_FIM)
[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open    Volume
Ticker       ITUB4.SA   ITUB4.SA   ITUB4.SA   ITUB4.SA  ITUB4.SA
Date                                                            
2019-04-09  21.084408  21.127626  20.751008  21.115278  14723610
2019-04-10  21.047365  21.263456  20.874490  21.238760  28947490
2019-04-11  20.559616  21.016497  20.442310  20.880668  13300540
2019-04-12  20.343521  20.880665  20.158300  20.374392  32403250
2019-04-15  20.331179  20.578141  20.152131  20.522575  18180470


Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Amplitude Diária (<i>Daily Range</i>)</b><br>
Calculada pela diferença aritmética entre a máxima e a mínima do dia (<i>H<sub>t</sub> − L<sub>t</sub></i>).  
Captura a <b>volatilidade intradiária absoluta</b>, servindo como proxy para a <b>incerteza</b> e a <b>agressividade</b> dos participantes do mercado durante o pregão, independentemente da direção do fechamento.

</div>

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Corpo Real (<i>Real Body</i>)</b><br>
Representa o deslocamento líquido do preço, calculado como (<i>C<sub>t</sub> − O<sub>t</sub></i>).  
Quantifica matematicamente a <b>força direcional</b> e a <b>convicção</b> do movimento: valores positivos indicam domínio comprador, enquanto valores negativos indicam domínio vendedor.  
Para o modelo, é essencial para diferenciar dias de <b>alta convicção</b> de períodos de <b>indecisão</b> (ex.: <i>Doji</i>).

</div>

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Sombras Superior e Inferior (<i>Upper & Lower Shadows</i>)</b><br>
Quantificam a <b>rejeição de preços</b> em extremos intradiários.  
A <b>Sombra Superior</b> (<i>H<sub>t</sub> − max(O<sub>t</sub>, C<sub>t</sub>)</i>) indica pressão vendedora em níveis elevados, enquanto a <b>Sombra Inferior</b> (<i>min(O<sub>t</sub>, C<sub>t</sub>) − L<sub>t</sub></i>) reflete defesa compradora em níveis baixos.  
São vitais para detectar <b>exaustão de tendência</b> e <b>armadilhas de liquidez</b>.

</div>


In [5]:
itau['Daily_Range'] = itau['High'] - itau['Low']
itau['Real_Body'] = itau['Close'] - itau['Open']
itau['Upper_Shadow'] = itau['High'] - itau[['Open', 'Close']].max(axis=1)
itau['Lower_Shadow'] = itau[['Open', 'Close']].min(axis=1) - itau['Low']
print(itau.head())

itau.columns

Price           Close       High        Low       Open    Volume  Daily_Range  \
Date                                                                            
2019-04-09  21.084408  21.127626  20.751008  21.115278  14723610     0.376618   
2019-04-10  21.047365  21.263456  20.874490  21.238760  28947490     0.388966   
2019-04-11  20.559616  21.016497  20.442310  20.880668  13300540     0.574187   
2019-04-12  20.343521  20.880665  20.158300  20.374392  32403250     0.722365   
2019-04-15  20.331179  20.578141  20.152131  20.522575  18180470     0.426009   

Price       Real_Body  Upper_Shadow  Lower_Shadow  
Date                                               
2019-04-09  -0.030870      0.012349      0.333399  
2019-04-10  -0.191395      0.024696      0.172875  
2019-04-11  -0.321052      0.135829      0.117306  
2019-04-12  -0.030871      0.506273      0.185221  
2019-04-15  -0.191396      0.055566      0.179047  


Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Range', 'Real_Body',
       'Upper_Shadow', 'Lower_Shadow'],
      dtype='object', name='Price')

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Log-Retorno (<i>Log Returns</i>)</b><br>
Diferença entre o logaritmo natural do preço atual e o anterior (<i>ln(P<sub>t</sub> / P<sub>t−1</sub>)</i>).  
É preferível ao retorno simples por favorecer a <b>estacionariedade</b> da série e a <b>simetria estatística</b> — movimentos de alta e baixa de mesma intensidade possuem magnitudes numéricas equivalentes.  
Essa propriedade facilita a <b>convergência</b> e a estabilidade de algoritmos baseados em <b>gradient boosting</b>.

</div>


In [6]:
itau['log_retorno'] = np.log(itau['Close']/itau['Close'].shift(1))
itau['Log_Retorno_t-1'] = itau['log_retorno'].shift(1)
print(itau['log_retorno'].head())
itau.columns

Date
2019-04-09         NaN
2019-04-10   -0.001758
2019-04-11   -0.023447
2019-04-12   -0.010566
2019-04-15   -0.000607
Name: log_retorno, dtype: float64


Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Range', 'Real_Body',
       'Upper_Shadow', 'Lower_Shadow', 'log_retorno', 'Log_Retorno_t-1'],
      dtype='object', name='Price')

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Janelas Deslizantes — Médias e Volatilidade (Rolling Statistics)</b><br>
Estatísticas aplicadas sobre janelas de <b>14, 30 e 60 períodos</b>.  
A <b>Média Móvel</b> captura a tendência central de curto e médio prazo, enquanto a <b>Volatilidade</b> (desvio padrão dos <i>log-retornos</i>) quantifica o risco histórico recente.  
Essa dualidade permite ao modelo contextualizar o preço atual dentro de diferentes <b>regimes de mercado</b> — <i>calmo/direcional</i> versus <i>volátil/errático</i>.

</div>


In [7]:
janelas = [14, 30, 60]

for n in janelas:
    itau[f'JanelaMedia{n}'] = itau['Close'].rolling(window=n).mean()

    itau[f'Volatilidade{n}'] = itau['log_retorno'].rolling(window=n).std() 

itau.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Range', 'Real_Body',
       'Upper_Shadow', 'Lower_Shadow', 'log_retorno', 'Log_Retorno_t-1',
       'JanelaMedia14', 'Volatilidade14', 'JanelaMedia30', 'Volatilidade30',
       'JanelaMedia60', 'Volatilidade60'],
      dtype='object', name='Price')

## II. Indicadores Técnicos e Momentum

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 10px;
">

<b>Média Móvel Simples (SMA — 7 e 21 períodos)</b><br>
Filtro linear que calcula a média aritmética dos preços de fechamento (<i>AdjClose</i>).  
Atua como um <b>suavizador de ruído de alta frequência</b>, permitindo ao modelo identificar a tendência prevalente.  
A janela de <b>7 períodos</b> captura o fluxo imediato (semanal), enquanto a de <b>21 períodos</b> representa o consenso mensal de valor.

</div>

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Média Móvel Exponencial (EMA — 12 e 26 períodos)</b><br>
Diferentemente da SMA, aplica um fator de ponderação recursivo que atribui maior peso aos dados mais recentes.  
É matematicamente mais <b>reativa</b> (<i>α = 2/(N + 1)</i>), permitindo detectar <b>reversões de tendência</b> com menor atraso (<i>lag</i>).  
Serve como base para indicadores derivados, sendo fundamental na construção do <b>MACD</b>.

</div>


In [8]:
itau['SMA_7'] = itau['Close'].rolling(window=7).mean()
itau['SMA_21'] = itau['Close'].rolling(window=21).mean()

itau['EMA_12'] =itau['Close'].ewm(span=12, adjust=False).mean()
itau['EMA_26'] =itau['Close'].ewm(span=26, adjust=False).mean()

itau.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Range', 'Real_Body',
       'Upper_Shadow', 'Lower_Shadow', 'log_retorno', 'Log_Retorno_t-1',
       'JanelaMedia14', 'Volatilidade14', 'JanelaMedia30', 'Volatilidade30',
       'JanelaMedia60', 'Volatilidade60', 'SMA_7', 'SMA_21', 'EMA_12',
       'EMA_26'],
      dtype='object', name='Price')

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>MACD (Moving Average Convergence Divergence)</b><br>
Oscilador calculado pela diferença entre a <b>EMA rápida (12)</b> e a <b>EMA lenta (26)</b>.  
Mede a <b>velocidade</b> e a <b>aceleração</b> da tendência.  
O modelo utiliza tanto a <b>linha MACD</b> quanto o seu <b>Sinal</b> (suavização de 9 períodos) para identificar <b>divergências</b> e o <b>momentum</b> do movimento atual em relação ao histórico.

</div>


In [9]:

itau['MACD_Line'] = itau['EMA_12'] - itau['EMA_26']
itau['MACD_Signal'] = itau['MACD_Line'].ewm(span=9, adjust=False).mean()

itau.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Range', 'Real_Body',
       'Upper_Shadow', 'Lower_Shadow', 'log_retorno', 'Log_Retorno_t-1',
       'JanelaMedia14', 'Volatilidade14', 'JanelaMedia30', 'Volatilidade30',
       'JanelaMedia60', 'Volatilidade60', 'SMA_7', 'SMA_21', 'EMA_12',
       'EMA_26', 'MACD_Line', 'MACD_Signal'],
      dtype='object', name='Price')

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>RSI (Relative Strength Index — 7 e 14 períodos)</b><br>
Indicador de <b>momentum</b> normalizado entre 0 e 100, calculado pela razão suavizada (método de Wilder) entre ganhos e perdas médios.  
Identifica condições de <b>sobrecompra</b> (&gt; 70) e <b>sobrevenda</b> (&lt; 30), além da velocidade da mudança dos preços, sinalizando potenciais pontos de <b>reversão à média</b>.

</div>


In [10]:

itau['Delta'] = itau['Close'].diff()
itau['Gain'] = itau['Delta'].clip(lower=0)
itau['Loss'] = itau['Delta'].clip(upper=0).abs()

periodos_rsi = [7, 14]

for n in periodos_rsi:
    avg_gain = itau['Gain'].ewm(alpha=1/n, min_periods=n, adjust=False).mean()
    avg_loss = itau['Loss'].ewm(alpha=1/n, min_periods=n, adjust=False).mean()
    
    rs = avg_gain / avg_loss
    
    itau[f'RSI_{n}'] = 100 - (100 / (1 + rs))
itau.drop(columns=['Delta', 'Gain', 'Loss'], inplace=True)

print(itau[['RSI_7', 'RSI_14']].tail())
itau.columns

Price           RSI_7     RSI_14
Date                            
2025-09-24  59.048428  59.213262
2025-09-25  52.041797  55.648759
2025-09-26  55.792599  57.343605
2025-09-29  60.017114  59.283910
2025-09-30  63.359289  60.852854


Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Range', 'Real_Body',
       'Upper_Shadow', 'Lower_Shadow', 'log_retorno', 'Log_Retorno_t-1',
       'JanelaMedia14', 'Volatilidade14', 'JanelaMedia30', 'Volatilidade30',
       'JanelaMedia60', 'Volatilidade60', 'SMA_7', 'SMA_21', 'EMA_12',
       'EMA_26', 'MACD_Line', 'MACD_Signal', 'RSI_7', 'RSI_14'],
      dtype='object', name='Price')

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>Bandas de Bollinger (21 períodos, 2 desvios)</b><br>
Envelope de volatilidade composto por uma média central e duas bandas externas (<i>μ ± 2σ</i>).  
Contextualizam o preço em termos estatísticos: toques nas bandas indicam desvios extremos da normalidade.  
Para o algoritmo, funcionam como medidas dinâmicas de <b>suporte</b>, <b>resistência</b> e <b>expansão de volatilidade</b>.

</div>


In [11]:
itau['BBmean'] = itau['Close'].rolling(window=21).mean()
itau['BBstd'] = itau['Close'].rolling(window=21).std()

itau['BBupper'] = itau['BBmean'] + (2* itau['BBstd'])
itau['BBLower'] = itau['BBmean'] - (2* itau['BBstd'])

itau.columns

Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Range', 'Real_Body',
       'Upper_Shadow', 'Lower_Shadow', 'log_retorno', 'Log_Retorno_t-1',
       'JanelaMedia14', 'Volatilidade14', 'JanelaMedia30', 'Volatilidade30',
       'JanelaMedia60', 'Volatilidade60', 'SMA_7', 'SMA_21', 'EMA_12',
       'EMA_26', 'MACD_Line', 'MACD_Signal', 'RSI_7', 'RSI_14', 'BBmean',
       'BBstd', 'BBupper', 'BBLower'],
      dtype='object', name='Price')

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>ATR (Average True Range)</b><br>
Mede a volatilidade absoluta considerando a amplitude do dia e os gaps de abertura em relação ao fechamento anterior.  
Essencial para modelagem de risco e definição de stops em pontos reais.

</div>


In [12]:

itau['Prev_Close'] = itau['Close'].shift(1)

itau['Range_1'] = itau['High'] - itau['Low']
itau['Range_2'] = (itau['High'] - itau['Prev_Close']).abs()
itau['Range_3'] = (itau['Low'] - itau['Prev_Close']).abs()

itau['TR'] = itau[['Range_1', 'Range_2', 'Range_3']].max(axis=1)
itau['ATR_7'] = itau['TR'].rolling(window=7).mean()
itau['ATR_14'] = itau['TR'].rolling(window=14).mean()

itau.drop(columns=['Prev_Close', 'Range_1', 'Range_2', 'Range_3'], inplace=True)
print(itau[['TR', 'ATR_7', 'ATR_14']].tail())
itau.columns

Price             TR     ATR_7    ATR_14
Date                                    
2025-09-24  0.406201  0.597831  0.635617
2025-09-25  0.519558  0.593783  0.613350
2025-09-26  0.358964  0.515511  0.595807
2025-09-29  0.651816  0.555997  0.603904
2025-09-30  0.736828  0.587036  0.619423


Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Range', 'Real_Body',
       'Upper_Shadow', 'Lower_Shadow', 'log_retorno', 'Log_Retorno_t-1',
       'JanelaMedia14', 'Volatilidade14', 'JanelaMedia30', 'Volatilidade30',
       'JanelaMedia60', 'Volatilidade60', 'SMA_7', 'SMA_21', 'EMA_12',
       'EMA_26', 'MACD_Line', 'MACD_Signal', 'RSI_7', 'RSI_14', 'BBmean',
       'BBstd', 'BBupper', 'BBLower', 'TR', 'ATR_7', 'ATR_14'],
      dtype='object', name='Price')

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>SQZ (Squeeze Momentum)</b><br>
Indicador híbrido que identifica períodos de compressão de volatilidade.  
Matematicamente, ocorre quando as <b>Bandas de Bollinger</b> estreitam-se em relação aos <b>Canais de Keltner</b> (baseados em ATR).  
Sinaliza regimes de <b>acumulação silenciosa</b> que estatisticamente precedem movimentos explosivos de preço.

</div>


In [13]:
itau['KC_Upper'] = itau['SMA_21'] + (1.5 * itau['ATR_14'])
itau['KC_Lower'] = itau['SMA_21'] - (1.5 * itau['ATR_14'])

condicao_teto = itau['BBupper'] < itau['KC_Upper']
condicao_chao = itau['BBLower'] > itau['KC_Lower']

itau['SQZ_On'] = (condicao_teto & condicao_chao).astype(int)
itau.drop(columns=['KC_Upper', 'KC_Lower'], inplace=True)

print(itau['SQZ_On'].value_counts())
itau.columns

SQZ_On
0    1303
1     311
Name: count, dtype: int64


Index(['Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Range', 'Real_Body',
       'Upper_Shadow', 'Lower_Shadow', 'log_retorno', 'Log_Retorno_t-1',
       'JanelaMedia14', 'Volatilidade14', 'JanelaMedia30', 'Volatilidade30',
       'JanelaMedia60', 'Volatilidade60', 'SMA_7', 'SMA_21', 'EMA_12',
       'EMA_26', 'MACD_Line', 'MACD_Signal', 'RSI_7', 'RSI_14', 'BBmean',
       'BBstd', 'BBupper', 'BBLower', 'TR', 'ATR_7', 'ATR_14', 'SQZ_On'],
      dtype='object', name='Price')

<div style="
    font-size: 0.85em;
    color: #c9c9c9;
    border-left: 3px solid #555;
    padding-left: 12px;
    margin-top: 8px;
">

<b>KDJ (Oscilador Estocástico Derivado)</b><br>
Refinamento do oscilador estocástico clássico, composto pelas linhas <b>K</b> (rápida), <b>D</b> (lenta) e <b>J</b> (divergência).  
O KDJ localiza o fechamento atual dentro do intervalo Máxima–Mínima recente.  
A linha <b>J</b>, mais sensível, permite antecipar pontos de virada de curto prazo com maior precisão que o estocástico tradicional.

</div>


In [14]:
itau['Low_14'] = itau['Low'].rolling(window=14).min()
itau['High_14'] = itau['High'].rolling(window=14).max()

itau['RSV'] = 100 * ((itau['Close'] - itau['Low_14']) / (itau['High_14'] - itau['Low_14']))

itau['K'] = itau['RSV'].ewm(com=2, adjust=False).mean()
itau['D'] = itau['K'].ewm(com=2, adjust=False).mean()
itau['J'] = (3 * itau['K']) - (2 * itau['D'])

itau.drop(columns=['Low_14', 'High_14', 'RSV'], inplace=True)

# Conferência final da Parte II
print("Colunas Finais da Parte II:")
print(itau.columns[-3:])

Colunas Finais da Parte II:
Index(['K', 'D', 'J'], dtype='object', name='Price')


Dado que usaremos o XGboost, devemos aplicar um alvo, nesse sentido, criamos 2: Um alvo de retorno(Retorno 5 dias) e um alvo binário (Retorno positivo ou não em 5 dias)

In [15]:
import numpy as np
import pandas as pd

itau_semanal = itau[itau.index.dayofweek == 4].copy()

itau_semanal.dropna(inplace=True)

itau_semanal['AlvoRetorno'] = itau_semanal['Close'].shift(-1) / itau_semanal['Close'] - 1
itau_semanal['Alvo'] = np.where(itau_semanal['AlvoRetorno'] > 0, 1, 0)

itau_semanal.dropna(subset=['Alvo', 'AlvoRetorno'], inplace=True)
itau_semanal['Alvo'] = itau_semanal['Alvo'].astype(int)

itau_semanal.index.to_series().to_csv('./dados/BaseA.csv', index=False, header=['Data'])

Seja o nosso rebalanceamento somente nas sextas feiras em um intervalo semanal, então filtramos somente os dados onde a data era sexta feira, a fim de evitar o ruido dos outros dias que só serviriam para prever 5D+, algo nao útil quando focamos na sexta.

Tratamento de feature

Coeficiente de Correlação de Pearson

In [16]:
featuresTest = [c for c in itau_semanal.columns if c not in ['Alvo', 'AlvoRetorno', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
X = itau_semanal[featuresTest]
y = itau_semanal['Alvo']

corr_matrix = X.corr().abs() #pearsson 

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print(f"Features detectadas com alta correlação (> 0.95): {len(drop)}")
print(drop)

itau_final = X.drop(columns=drop)
print("Shape final das features após Pearson:", itau_final.shape)

Features detectadas com alta correlação (> 0.95): 12
['JanelaMedia30', 'JanelaMedia60', 'SMA_7', 'SMA_21', 'EMA_12', 'EMA_26', 'MACD_Signal', 'BBmean', 'BBupper', 'BBLower', 'TR', 'D']
Shape final das features após Pearson: (307, 19)


In [17]:
itau_final.columns

Index(['Daily_Range', 'Real_Body', 'Upper_Shadow', 'Lower_Shadow',
       'log_retorno', 'Log_Retorno_t-1', 'JanelaMedia14', 'Volatilidade14',
       'Volatilidade30', 'Volatilidade60', 'MACD_Line', 'RSI_7', 'RSI_14',
       'BBstd', 'ATR_7', 'ATR_14', 'SQZ_On', 'K', 'J'],
      dtype='object', name='Price')

In [18]:
mi_scores = mutual_info_classif(itau_final, y, random_state=42)
mi_series = pd.Series(mi_scores, index=itau_final.columns).sort_values(ascending=False)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(itau_final)
chi2_scores, p_values = chi2(X_scaled, y)
chi2_series = pd.Series(chi2_scores, index=itau_final.columns).sort_values(ascending=False)

print("TOP 10 Features por Informação Mútua")
print(mi_series.head(10))

print("\nTOP 10 Features por Qui-Quadrado")
print(chi2_series.head(10))

TOP 10 Features por Informação Mútua
Price
ATR_7              0.054490
Daily_Range        0.053543
log_retorno        0.048731
SQZ_On             0.043991
RSI_7              0.036605
Log_Retorno_t-1    0.024079
K                  0.022948
RSI_14             0.022775
Upper_Shadow       0.016128
Volatilidade14     0.014256
dtype: float64

TOP 10 Features por Qui-Quadrado
Price
SQZ_On            0.254139
RSI_7             0.242001
J                 0.231680
K                 0.168914
Daily_Range       0.158477
RSI_14            0.108207
Volatilidade14    0.095076
ATR_14            0.089461
ATR_7             0.079883
BBstd             0.043395
dtype: float64


In [19]:
lim = 0.01 

featSelect = chi2_series[chi2_series > lim].index.tolist()
featDrop = chi2_series[chi2_series <= lim].index.tolist()

itau_otimizado = itau_final[featSelect].copy()

print(f"Critério de Corte: Score > {lim}")
print(f"Total Original: {len(chi2_series)} variáveis")
print(f"Total Selecionado: {len(featSelect)} variáveis")
print("-" * 30)

print("\n VARIÁVEIS DESCARTADAS (Ruído):")
print(featDrop)

print("\n VARIÁVEIS MANTIDAS (Input do Modelo):")
print(itau_otimizado.columns.tolist())

print(itau_otimizado.describe().loc[['count', 'mean']])

Critério de Corte: Score > 0.01
Total Original: 19 variáveis
Total Selecionado: 16 variáveis
------------------------------

 VARIÁVEIS DESCARTADAS (Ruído):
['Volatilidade60', 'log_retorno', 'MACD_Line']

 VARIÁVEIS MANTIDAS (Input do Modelo):
['SQZ_On', 'RSI_7', 'J', 'K', 'Daily_Range', 'RSI_14', 'Volatilidade14', 'ATR_14', 'ATR_7', 'BBstd', 'Volatilidade30', 'Log_Retorno_t-1', 'JanelaMedia14', 'Upper_Shadow', 'Lower_Shadow', 'Real_Body']
Price     SQZ_On       RSI_7           J           K  Daily_Range     RSI_14  \
count  307.00000  307.000000  307.000000  307.000000   307.000000  307.00000   
mean     0.19544   51.855228   51.593367   52.019457     0.495592   52.21638   

Price  Volatilidade14      ATR_14       ATR_7      BBstd  Volatilidade30  \
count      307.000000  307.000000  307.000000  307.00000      307.000000   
mean         0.016948    0.540912    0.542228    0.62065        0.017211   

Price  Log_Retorno_t-1  JanelaMedia14  Upper_Shadow  Lower_Shadow   Real_Body  
count 

In [20]:
itau_otimizado.columns

Index(['SQZ_On', 'RSI_7', 'J', 'K', 'Daily_Range', 'RSI_14', 'Volatilidade14',
       'ATR_14', 'ATR_7', 'BBstd', 'Volatilidade30', 'Log_Retorno_t-1',
       'JanelaMedia14', 'Upper_Shadow', 'Lower_Shadow', 'Real_Body'],
      dtype='object', name='Price')

Filtramos as variáveis com alta correlação via Pearson e validamos sua importância através dos testes de Qui-Quadrado e Informação Mútua. Observou-se que as métricas estão consistentemente distantes de zero (o que indicaria independência ou ruído), confirmando que estas variáveis possuem sinal suficiente para contribuir com o desempenho do modelo.

In [21]:
print(itau_otimizado.isna().any())

Price
SQZ_On             False
RSI_7              False
J                  False
K                  False
Daily_Range        False
RSI_14             False
Volatilidade14     False
ATR_14             False
ATR_7              False
BBstd              False
Volatilidade30     False
Log_Retorno_t-1    False
JanelaMedia14      False
Upper_Shadow       False
Lower_Shadow       False
Real_Body          False
dtype: bool


In [22]:
itau_otimizado.to_csv('./dados/teste.csv')

Automatizando para todas as ações

In [ ]:
ativos = pd.read_csv('ibovespa.csv')